In [1]:
import pandas as pd

from google.colab import auth
auth.authenticate_user()

project_id='My First Project'
!gsutil cp -r gs://grab-ai-for-sea/preprocessedAccel_Sequences.pickle /content/
!gsutil cp -r gs://grab-ai-for-sea/preprocessedAccel_Labels.pickle /content/

Copying gs://grab-ai-for-sea/preprocessedAccel_Sequences.pickle...
/ [1 files][  1.2 GiB/  1.2 GiB]  101.3 MiB/s                                   
Operation completed over 1 objects/1.2 GiB.                                      
Copying gs://grab-ai-for-sea/preprocessedAccel_Labels.pickle...
/ [1 files][781.6 KiB/781.6 KiB]                                                
Operation completed over 1 objects/781.6 KiB.                                    


In [0]:
import pickle

pickle_in = open("preprocessedAccel_Sequences.pickle", "rb")
data = pickle.load(pickle_in)

pickle_in_y = open("preprocessedAccel_Labels.pickle", "rb")
labels = pickle.load(pickle_in_y)

In [3]:
import numpy as np
import random

print(data.shape)
print(labels.shape)

split = 0.8

train_x = data[:int(float(len(data))*split)]
train_y = labels[:int(float(len(labels))*split)]

validation_x = data[int(float(len(data))*split)+1:]
validation_y = labels[int(float(len(labels))*split)+1:]

print(len(train_x), len(validation_x))

(100020, 778, 4)
(100020,)
80016 20003


In [4]:
import time # for naming

# create more constants
EPOCHS = 30
BATCH_SIZE = 64

# import all the tensorflow stuff we need
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, Flatten, BatchNormalization # batch normalization layer normalizes data
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint # all the callbacks we will use
from tensorflow import keras
from tensorflow.keras.models import load_model

!mkdir models

print("input size is: ", train_x.shape[1:])
# ===================================== Model ============================================

lstm_layers = [4] # only need to continue training from second iteration of LSTM layer here.
nodes = [128] # try higher node number
batch_size = [64] # try decreasing batch size, and increasing datapoints
decayrate = 1e-6
learningrate = 0.0001
dropout = 0.3

for node_num in nodes:
    for layers in lstm_layers:
        for bs in batch_size:
            model = Sequential()
            NAME = f"{layers+1}-lstm-{node_num}-nodes-{bs}-batch-{EPOCHS}-epochs-{dropout}-dropout-{int(time.time())}"
            print("Training: ", NAME)
            for i in range(layers):
                model.add(CuDNNLSTM(node_num, input_shape=(None, train_x.shape[2]), return_sequences=True)) # 128 nodes on this layer
                model.add(Dropout(dropout))
                model.add(BatchNormalization())

            model.add(CuDNNLSTM(node_num, input_shape=(None, train_x.shape[2]))) # can also try changing the number of nodes here... 
            model.add(Dropout(0.2))
            model.add(BatchNormalization())

            model.add(Dense(32, activation="relu"))
            model.add(Dropout(0.2))

            model.add(Dense(2, activation="softmax")) # final layer.

            # specify the optimizer
            opt = tf.keras.optimizers.Adam(lr=learningrate, decay=decayrate)

    
            model.compile(loss='sparse_categorical_crossentropy',
                       optimizer=opt,
                       metrics=['accuracy'])

            tensorboard = TensorBoard(log_dir=f"logs/{NAME}")

            history = model.fit(np.array(train_x), np.array(train_y),
                                 batch_size=bs,
                                 epochs=EPOCHS,
                                 validation_data=(np.array(validation_x), np.array(validation_y)),
                                 callbacks = [tensorboard])
            !gsutil cp -r /content/logs/{NAME} gs://grab-ai-for-sea/logs
            print("copied logs to Google Cloud Storage")

mkdir: cannot create directory ‘models’: File exists


W0616 05:22:41.329031 139727582562176 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


input size is:  (778, 4)
Training:  5-lstm-128-nodes-64-batch-20-epochs-0.3-dropout-1560662561
Train on 80016 samples, validate on 20003 samples
Epoch 1/20
   64/80016 [..............................] - ETA: 2:03:37 - loss: 1.0502 - acc: 0.5000

W0616 05:22:53.403532 139727582562176 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (0.411164). Check your callbacks.


80016/80016 [==============================] - 365s 5ms/sample - loss: 0.7885 - acc: 0.5309 - val_loss: 0.6848 - val_acc: 0.5632
Epoch 2/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.7028 - acc: 0.5526 - val_loss: 0.6713 - val_acc: 0.5901
Epoch 3/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.6819 - acc: 0.5687 - val_loss: 0.6640 - val_acc: 0.6000
Epoch 4/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.6714 - acc: 0.5840 - val_loss: 0.6608 - val_acc: 0.5992
Epoch 5/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.6665 - acc: 0.5938 - val_loss: 0.6631 - val_acc: 0.5901
Epoch 6/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.6623 - acc: 0.5966 - val_loss: 0.6550 - val_acc: 0.6079
Epoch 7/20
80016/80016 [==============================] - 360s 4ms/sample - loss: 0.6596 - acc: 0.6015 - val_loss: 0.6654 - val_acc: 0.5944
Epoch 8/20
80016/80016 [=======

In [6]:
index = 15

print(model.predict_classes(np.array([validation_x[index]])))
print(validation_y[index])

model.save("grab_ai_safety_accel.h5")

[1]
1


In [8]:
!gsutil cp /content/grab_ai_safety_accel.h5 gs://grab-ai-for-sea

Copying file:///content/grab_ai_safety_accel.h5 [Content-Type=application/octet-stream]...
/ [1 files][  7.0 MiB/  7.0 MiB]                                                
Operation completed over 1 objects/7.0 MiB.                                      
